In [1]:
import pyodbc
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings("ignore")

In [2]:
def df_cluster(nits_clientes, distrito, fecha_inicial, fecha_final):
    # Conexion al dwh
    cnxn = pyodbc.connect(
        driver='{SQL Server}',
        server='192.168.100.58',
        uid='bilectura',
        pwd='D1sp@p3l3s')
    cursor = cnxn.cursor()

    df_SQL_nits = pd.DataFrame()

    for nit in nits_clientes:
        #Consulta SQL
        consulta_SQL = f"SELECT A.VTANIT AS 'Nitcliente', B.NITNOM AS 'Nombre cliente', A.VTAPRDCOD AS 'Codigo producto', SUM(A.VTAUNDINV) AS 'Unidades' FROM V_VTA_VTAHEC A LEFT JOIN DIM_NITS B ON A.VTANIT = B.NITNIT WHERE A.VTANIT = '{nit}' AND A.VTAFCH BETWEEN '{fecha_inicial}' AND '{fecha_final}' AND A.DSTCOD = {distrito} GROUP BY A.VTANIT, B.NITNOM, A.VTAPRDCOD HAVING SUM(A.VTAUNDINV) > 0"

        #Carga de la data desde el dwh de Dispapeles y se guarda en df
        cursor.execute(consulta_SQL)
        rows = cursor.fetchall()
        df_SQL_int = pd.DataFrame.from_records(rows, columns=[col[0] for col in cursor.description])
        df_SQL_int["Unidades"] = df_SQL_int["Unidades"].astype(int)

        df_SQL_nits = pd.concat([df_SQL_nits, df_SQL_int], ignore_index= True)

    # df_SQL = df_SQL_nits.groupby("Fecha").sum().reset_index()
    df_SQL_nits
    df_productos = df_SQL_nits.groupby("Codigo producto").sum().reset_index()
    df_productos = df_productos.drop("Unidades", axis= 1)
    df_nits = df_SQL_nits.groupby(["Nitcliente", "Nombre cliente"]).sum().reset_index()
    df_nits = df_nits.drop("Unidades", axis= 1)

    return df_SQL_nits, df_productos, df_nits

In [37]:
def nombre_cliente(df, nit):
    a = df[df["Nitcliente"] == nit]["Nombre cliente"].values
    
    return a[0]


In [3]:
df_productos = pd.read_excel("C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Databases/Maestra gerencias.xlsx", sheet_name= "Maestra productos BI")
col = ["PRDCOD", "PRDNOM", "Gerente", "Línea CAM"]
df_productos = df_productos[col]
df_productos.columns = ["Codigo producto", "Nombre producto", "Gerencia", "Linea CAM"]

In [4]:
df_clusters = pd.read_csv("C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Clustering/Clustering 12-04-23.csv",
                            encoding= 'utf-8', decimal= ",", sep= ";")
col_eliminar = ["Escala R", "Escala M", "Escala F", "Distrito-Nombretipozona", "Cluster"]
df_clusters = df_clusters.drop(col_eliminar, axis= 1)

filtro_distrito = 10
filtro_tipozona = "Artes graficas"
filtro_cluster = "A"

df_clusters_f = df_clusters[
                            (df_clusters["Codigo distrito"] == filtro_distrito) &
                            (df_clusters["Nombre tipo zona"] == filtro_tipozona) &
                            (df_clusters["Letra cluster"] == filtro_cluster)
                            ]

In [5]:
# Variables auxiliares
lista_nits = []
for nit_suc in df_clusters_f["Nit cliente-sucursal"].tolist():
    lista_nits.append(nit_suc.split("-")[0])

fecha_final = '2023-03-31'
fecha_inicial = '2022-04-01'
fecha_final = datetime.strptime(fecha_final, '%Y-%m-%d').strftime('%Y-%m-%d')
fecha_inicial = datetime.strptime(fecha_inicial, '%Y-%m-%d').strftime('%Y-%m-%d')

In [6]:
#dfs para EDA y forecast
ventas_cluster, ventas_productos, ventas_nits = df_cluster(nits_clientes= lista_nits, distrito= filtro_distrito, fecha_inicial= fecha_inicial, fecha_final= fecha_final)

In [8]:
productos_cluster = pd.merge(ventas_productos, df_productos, on= "Codigo producto", how= "left")

### Filtro colaborativo basado en usuario:

In [10]:
# Crear una matriz de utilidad de usuarios y películas
utility_matrix = ventas_cluster.pivot(index='Nitcliente', columns='Codigo producto', values='Unidades').fillna(0)
# Calcular la similitud del coseno entre los usuarios
user_similarity = cosine_similarity(utility_matrix)

In [11]:
# Función para obtener las películas recomendadas para un usuario específico
def get_user_recommendations(Nitcliente, num_recommendations=10):
    # Obtener la fila de similitud del usuario
    index = utility_matrix.index.tolist()
    user_similarities = user_similarity[index.index(Nitcliente)]

    # Calcular el promedio ponderado de las clasificaciones de películas de los usuarios similares
    weighted_productos = np.dot(user_similarities, utility_matrix.values) / np.sum(np.abs(user_similarities))

    # Excluir las películas que el usuario ya ha visto
    producto_comprado = utility_matrix.loc[Nitcliente] > 0
    weighted_productos[producto_comprado] = 0

    # Ordenar las películas recomendadas por la clasificación ponderada
    codigos_productos = np.argsort(weighted_productos)[::-1][:num_recommendations+1]

    # Obtener los títulos de las películas recomendadas
    recommended_productos = productos_cluster.iloc[productos_cluster.index.isin(codigos_productos)]

    return recommended_productos



In [36]:
a = ventas_nits[ventas_nits["Nitcliente"] == 800170615]["Nombre cliente"].values
a[0]

'FOTOLITO HEL LTDA.                      '

In [38]:
Nitcliente = 800170615
Nombre_cliente = nombre_cliente(ventas_nits, Nitcliente)
recommended_productos = get_user_recommendations(Nitcliente= Nitcliente, num_recommendations= 5)
print(f"Productos recomendados para el cliente {Nitcliente} {Nombre_cliente}:")
recommended_productos

Productos recomendados para el cliente 800170615 FOTOLITO HEL LTDA.                      :


,Codigo producto,Nombre producto,Gerencia,Linea CAM
0,20,BOND BLANCO,Papel para escritura,Bond
41,839,REPROGRAF A.B. RESMILLA OFICIO,Resmilla,Resmilla - Reprograf
42,1000,PAPEL PERIODICO,Periódico,Periódico
79,5995,FASSON ESM80 SEGURIDAD S1600 SCK 85,Papeles especiales,Autoadhesivos
118,10675,REPROGRAF NATURAL CARTA,Resmilla,Resmilla - Natural
341,50600,PROPALCOTE C2S GLOSS-70.5 Y 141,Papel para escritura,Esmaltado
